In [102]:
using Agents
using BenchmarkTools

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1260


In [2]:
# Define the agent.  Contains an id, position (2d space), and wealth
mutable struct WealthAgent <: AbstractAgent
    id::Int
    pos::NTuple{2,Int}
    wealth::Int
end

In [ ]:
#Initialization function for the model
function wealth_model(; dims=(25, 25), numagents=100, wealth=1)
    space = GridSpace(dims, periodic=true, moore=true)
    model = ABM(WealthAgent, space, scheduler=random_activation)
    for i in 1:numagents
        add_agent!(model, wealth)
    end
    return model
end

In [44]:
#Define function for moving the agent - can move one grid square (diagonally included)
function agent_move!(agent, model)
    node = Agents.coord2vertex(agent.pos, model)
    neighbors = node_neighbors(node, model)
    target = rand(neighbors)
    move_agent!(agent, target, model)
end

agent_move! (generic function with 1 method)

In [40]:
#Define function for passing wealth from one agent to another random agent within the same grid square
function agent_give!(agent, model)
    cellmates = Agents.get_node_contents(agent, model)
    if size(cellmates)[1] > 1
        other = model[rand(cellmates)]
        other.wealth += 1
        agent.wealth -= 1
    end
end

agent_give! (generic function with 1 method)

In [31]:
#Define function for updating an agent
function agent_step!(agent, model)
    agent_move!(agent, model)
    if agent.wealth > 0
        agent_give!(agent, model)
    end
end

agent_step! (generic function with 1 method)

In [91]:
#Define function for calculating entropy (Gini)
# a is a generator of agents
function gini(a)
    N = Agents.nagents(model)
    w = Array{Int64, 1}(undef, N)
    for (i, agent) in enumerate(collect(a)) 
        w[i] = agent
    end
    x = sort(w)
    B = 0
    for i in 1:N
        B += x[i]*(N-i)
    end
    B /= (N * sum(x))
    return (1 + (1/N) - 2*B) 
end

gini (generic function with 1 method)

In [107]:
#Use benchmark library to test model speed
@benchmark begin
model = wealth_model(dims=(100, 100), numagents=10000)
adata = [:wealth, :pos]
data, _ = run!(model, agent_step!, 200; adata=adata)
end

BenchmarkTools.Trial: 
  memory estimate:  355.81 MiB
  allocs estimate:  6133379
  --------------
  minimum time:     1.690 s (2.60% GC)
  median time:      1.775 s (10.53% GC)
  mean time:        1.756 s (8.82% GC)
  maximum time:     1.804 s (10.36% GC)
  --------------
  samples:          3
  evals/sample:     1